In [68]:
import pandas as pd
import random as rd
import datetime as dt
import pickle,glob,re

In [69]:
cumule=True
#repNom="/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/"
repNom="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/"
#repNomSortie="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
repNomSortie="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
tirageDepart="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-49-T500000-F31170.pkl"

In [70]:
listeTirages=glob.glob(repNom+"*-Tirage.pkl")

In [71]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [72]:
compositionLongitudinale=[("1Ko",10),("10Ko",9),("100Ko",9),("1Mo",19)]
compositionLongitudinale=[("10Mo",20)]
compositionLongitudinale=[("10Ko",10),("100Ko",9),("1Mo",9)]
compositionLongitudinale=[("100Ko",6)]

In [73]:
serieLongitudinale=[]
for serie,nbSerie in compositionLongitudinale:
    for nom in rd.sample(tiragesNom(serie),nbSerie):
        m=re.match(ur"^.*-%s-(\d{2})-T.*-F.*-Tirage.*"%serie,nom)
        if m:
#            print m.group(1), nom
            serieLongitudinale.append(nom)
        else:
            print "mauvais nom de fichier",nom
serieLongitudinale

['/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-16-T100000-F14845-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-26-T100000-F14857-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-15-T100000-F14718-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-29-T100000-F14954-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-38-T100000-F14871-Tirage.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/MGC-100Ko-05-T100000-F14872-Tirage.pkl']

In [74]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [79]:
indexLexique=lireLexique(serieLongitudinale[0])
indexLexique["tir1"]=0
nTirageDepart=0
if tirageDepart:
    lexique,cumulLexique=lireLexique(tirageDepart).align(indexLexique)
    cumulLexique["tir1"]=cumulLexique["tir1"].fillna(value=0)+lexique["tir1"].fillna(value=0)
    nTirageDepart=int(tirageDepart.rsplit("/")[-1].split("-")[1])+1

In [80]:
cumulLexique["tir1"].sum()

500000.0

In [77]:
nTirageDepart

50

In [81]:
timeStamp=dt.datetime.now().strftime("%y%m%d-%H%M")
for nTirage,tirage in enumerate(serieLongitudinale):
    lexique=lireLexique(tirage)
    if cumule:
        cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    else:
        cumulLexique["tir1"]=lexique["tir1"]
    print "(%d,%d),"%(cumulLexique["tir1"].sum(),cumulLexique["tir1"].count())
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)
    sortieNom=repNomSortie+"Longitudinal-%02d-T%d-F%d.pkl"%(nTirage+nTirageDepart,nbTokens,nbTypes)
    with open(sortieNom,"wb") as output:
        pickle.dump(lexique1, output, pickle.HIGHEST_PROTOCOL)

(600000,328098),
(600000,33424),
(700000,328098),
(700000,35357),
(800000,328098),
(800000,36975),
(900000,328098),
(900000,38562),
(1000000,328098),
(1000000,39898),
(1100000,328098),
(1100000,41134),
